In [ ]:
import time
import numpy as np
import rasterio
from pathlib import Path
import cv2
import warnings
from concurrent.futures import ThreadPoolExecutor
import multiprocessing as mp
warnings.filterwarnings('ignore')

In [ ]:
CONFIG = {
    'sentinel2_composites_dir': './satellite_data/processed/monthly_cloud_free/',
    'output_dir': './false_color_images/',
    'highlight_color': [0, 255, 0],
    'highlight_alpha': 0.6,
    'ndvi_threshold': 0.2,
    'max_workers': min(8, mp.cpu_count()),
}

In [ ]:
class FalseColorGenerator:
    """Generate false color infrared images with vegetation highlighting."""
    
    def __init__(self):
        self.output_dir = Path(CONFIG['output_dir'])
        self.output_dir.mkdir(parents=True, exist_ok=True)
        self.ndvi_threshold = CONFIG['ndvi_threshold']
    
    def read_composite_bands(self, image_path):
        """Read multi-band composite and extract required bands."""
        try:
            with rasterio.open(image_path) as src:
                data = src.read().astype(np.float32)
                
                # Extract bands: B02(blue), B03(green), B04(red), B08(nir)
                if data.shape[0] >= 4:
                    bands = {
                        'blue': data[0],   # B02
                        'green': data[1],  # B03  
                        'red': data[2],    # B04
                        'nir': data[3]     # B08
                    }
                    return bands
                
        except Exception as e:
            print(f"Error reading {image_path}: {e}")
            return None
    
    def create_false_color_infrared(self, red, green, blue, nir):
        """Create false color infrared image (NIR-Red-Green)."""
        try:
            # False color infrared: NIR as Red, Red as Green, Green as Blue
            rgb_stack = np.stack([nir, red, green], axis=-1)
            
            # Handle NaN values
            valid_mask = ~np.isnan(rgb_stack).any(axis=2)
            if not np.any(valid_mask):
                return None
            
            # Scale from Sentinel-2 reflectance to 0-1
            rgb_normalized = np.clip(rgb_stack / 10000.0, 0, 1)
            
            # Apply contrast stretching
            rgb_enhanced = np.zeros_like(rgb_normalized, dtype=np.float32)
            
            for i in range(3):
                channel_data = rgb_normalized[:, :, i]
                valid_data = channel_data[valid_mask]
                
                if len(valid_data) > 0:
                    p1, p99 = np.percentile(valid_data, [1, 99])
                    if p99 > p1:
                        stretched = (channel_data - p1) / (p99 - p1)
                        rgb_enhanced[:, :, i] = np.clip(stretched, 0, 1)
                    else:
                        rgb_enhanced[:, :, i] = channel_data
            
            # Apply gamma correction
            rgb_gamma = np.power(rgb_enhanced, 0.7)
            
            # Convert to 8-bit
            rgb_8bit = (rgb_gamma * 255).astype(np.uint8)
            
            return rgb_8bit
            
        except Exception as e:
            print(f"Error creating false color image: {e}")
            return None
    
    def detect_vegetation_ndvi(self, red, green, blue, nir):
        """Detect vegetation using configurable NDVI threshold."""
        try:
            eps = 1e-8
            
            # Calculate NDVI
            ndvi = (nir - red) / (nir + red + eps)
            
            # Vegetation detection using configurable threshold
            vegetation_mask = ndvi > self.ndvi_threshold
            
            # Calculate statistics
            total_pixels = vegetation_mask.size
            veg_pixels = np.sum(vegetation_mask)
            veg_percentage = (veg_pixels / total_pixels * 100) if total_pixels > 0 else 0
            
            print(f"    NDVI range: {np.nanmin(ndvi):.3f} to {np.nanmax(ndvi):.3f}")
            print(f"    Vegetation (NDVI > {self.ndvi_threshold}): {veg_pixels} pixels ({veg_percentage:.1f}%)")
            
            return vegetation_mask
            
        except Exception as e:
            print(f"Error calculating NDVI: {e}")
            return np.zeros((red.shape[0], red.shape[1]), dtype=bool)
    
    def apply_vegetation_highlight(self, false_color_image, vegetation_mask):
        """Apply green highlighting to vegetation areas."""
        try:
            highlighted_image = false_color_image.copy()
            
            if np.any(vegetation_mask):
                # Create green overlay
                overlay = np.zeros_like(highlighted_image)
                overlay[vegetation_mask] = CONFIG['highlight_color'][::-1]  # BGR to RGB
                
                # Blend overlay with original image
                alpha = CONFIG['highlight_alpha']
                highlighted_image[vegetation_mask] = cv2.addWeighted(
                    highlighted_image[vegetation_mask], 
                    1 - alpha,
                    overlay[vegetation_mask],
                    alpha, 
                    0
                )
            
            return highlighted_image
            
        except Exception as e:
            print(f"Error applying highlights: {e}")
            return false_color_image
    
    def process_image(self, image_path):
        """Process single image to create highlighted false color infrared."""
        try:
            # Check if output file already exists
            output_path = self.output_dir / f"{image_path.stem}_false_color_highlighted.png"
            if output_path.exists():
                print(f"⏭️ Skipping {image_path.name} (already processed)")
                return
            
            bands = self.read_composite_bands(image_path)
            if not bands or len(bands) < 4:
                print(f"✗ Insufficient bands in {image_path.name}")
                return
            
            blue = bands['blue']
            green = bands['green'] 
            red = bands['red']
            nir = bands['nir']
            
            # Create false color infrared image
            false_color_image = self.create_false_color_infrared(red, green, blue, nir)
            if false_color_image is None:
                print(f"✗ Failed to create false color image for {image_path.name}")
                return
            
            # Detect vegetation using configurable NDVI threshold
            vegetation_mask = self.detect_vegetation_ndvi(red, green, blue, nir)
            veg_pixels = np.sum(vegetation_mask)
            total_pixels = vegetation_mask.size
            veg_percentage = (veg_pixels / total_pixels * 100) if total_pixels > 0 else 0
            
            # Apply vegetation highlighting
            highlighted_image = self.apply_vegetation_highlight(false_color_image, vegetation_mask)
            
            # Convert to BGR for OpenCV and save
            highlighted_bgr = highlighted_image[..., ::-1]  # RGB to BGR
            
            output_path = self.output_dir / f"{image_path.stem}_false_color_highlighted.png"
            success = cv2.imwrite(str(output_path), highlighted_bgr)
            
            if success:
                print(f"✓ {image_path.name} -> {veg_percentage:.1f}% vegetation highlighted")
            else:
                print(f"✗ Failed to save {image_path.name}")
            
        except Exception as e:
            print(f"✗ Error processing {image_path.name}: {e}")
    
    def run_parallel(self):
        """Run processing in parallel."""
        composites_dir = Path(CONFIG['sentinel2_composites_dir'])
        if not composites_dir.exists():
            print(f"Directory not found: {composites_dir}")
            return
            
        files = list(composites_dir.glob("*_cloud_free.tif"))
        if not files:
            files = list(composites_dir.glob("*.tif")) + list(composites_dir.glob("*.tiff"))
        
        if not files:
            print(f"No composite files found in {composites_dir}")
            return
        
        # Count existing files to show progress
        existing_count = 0
        for file in files:
            output_path = self.output_dir / f"{file.stem}_false_color_highlighted.png"
            if output_path.exists():
                existing_count += 1
        
        print(f"🚀 Processing {len(files)} files to create highlighted false color infrared images")
        print(f"📁 Output: {self.output_dir}")
        print(f"🌱 Vegetation detection: NDVI > {self.ndvi_threshold}")
        if existing_count > 0:
            print(f"⏭️ Skipping {existing_count} already processed files")
        
        with ThreadPoolExecutor(max_workers=CONFIG['max_workers']) as executor:
            executor.map(self.process_image, files)
        
        print(f"✅ Complete! Highlighted false color infrared images saved to: {self.output_dir}")

In [ ]:
def main():
    start_time = time.time()
    
    print("🎨 False Color Infrared Generator with Vegetation Highlighting")
    print("=" * 60)
    
    generator = FalseColorGenerator()
    generator.run_parallel()
    
    duration = time.time() - start_time
    print(f"⚡ Completed in {duration:.1f}s")

In [ ]:
if __name__ == "__main__":
    main()